In [1]:
import constants
cache_folder_name = 'test'
seed = 44

In [2]:
import autograd.numpy as np
np.random.seed(seed)
import python_utils.python_utils.basic as basic
import python_utils.python_utils.caching as caching
log_folder = '%s/%s' % (constants.cache_folder, cache_folder_name)
caching.init(constants.cache_folder, constants.which_hash, basic.archiver(log_folder))
%load_ext autoreload
%autoreload 2
import matplotlib
matplotlib.use('Agg')

[Errno 17] File exists: '/Users/fultonw/Documents/projects/runs/domain_adapt_run/domain_adapt_run/notebooks/new/cross_validation_multiple_fitters/v_data/cache/test'


In [3]:
import matplotlib.pyplot as plt
from IPython.display import display_pretty, display_html
import python_utils.python_utils.basic as basic
import pandas as pd
import pdb
import itertools
import functools
import scipy.stats
import scipy
import cvxopt
import autograd
import autograd.numpy as np
import domain_adapt.domain_adapt.new.fxns as fxns
import domain_adapt.domain_adapt.new.utils as utils
import domain_adapt.domain_adapt.new.optimizers as optimizers
import domain_adapt.domain_adapt.new.constructors as constructors
import domain_adapt.domain_adapt.new.lsif_constructor as lsif_constructor
import domain_adapt.domain_adapt.new.cv as cv
#import domain_adapt.domain_adapt.SDR_data as data
import domain_adapt_run.domain_adapt_run.notebooks.new.cross_validation_multiple_fitters.run as run
#import data.domain_adapt_data.domain_adapt_data.fxns as data
import domain_adapt.domain_adapt.new.data as data

Couldn't import dot_parser, loading of dot files will not be possible.


In [4]:
# define data getter
def random_data_getter(seed):
    np.random.seed(seed)
    xs_train, xs_test, ys_train, ys_test = data.v_data(
        num_train=100, 
        num_test=50, 
        useful_dim=1, 
        same_dim=10, 
        diff_dim=100
    )
    return xs_train, xs_test, ys_train, ys_test

In [5]:
# define evaluation stuff
eval_num_trials = 70
#eval_num_trials = 10
#eval_which_loss = 'square'
eval_which_loss = 'abs'
#eval_num_folds = 0
eval_num_folds = 3
eval_use_test = True

In [6]:
# define my fitter
cv_which_loss = 'square'
#cv_num_folds = 3
cv_num_folds = 0
#cv_use_test = False
cv_use_test = True
predictor_which_loss = 'square'
#predictor_which_loss = 'logistic'
num_basis = 100
u_dim = 1
pymanopt_options={'logverbosity':1, 'maxiter':10}
many_optimizer_num_cycles = 1
num_tries = 2
B_init_f_getter = lambda xs_train, ys_train, xs_test: (lambda: np.random.normal(size=(xs_train.shape[1],u_dim)))
tradeoff_weight_regs = 2**np.linspace(-3,3,7)
#tradeoff_weight_regs = [0.]
#tradeoff_weight_regs = 2**np.linspace(-1,1,3)
plot_b_info = constructors.simple_plot_b_info
#plot_b_info = None
fitter_list = [lsif_constructor.lsif_ratio_fitter(predictor_which_loss, num_basis, tradeoff_weight_reg, B_init_f_getter, num_tries=num_tries, pymanopt_options=pymanopt_options, many_optimizer_num_cycles=many_optimizer_num_cycles, plot_b_info=plot_b_info) for tradeoff_weight_reg in tradeoff_weight_regs]
my_fitter = cv.cv_fitter(fitter_list, cv_which_loss, cv_num_folds, cv_use_test)

In [7]:
# define naive weighted fitter
#cv_which_loss = 'square'
#cv_num_folds = 3
#cv_num_folds = 2
#cv_use_test = False
predictor_which_loss = 'square'
num_basis = 100
#u_dim = 8
#pymanopt_options={'logverbosity':1, 'maxiter':10}
many_optimizer_num_cycles = 1
num_tries = 1
B_init_f_getter = None
learn_projection = False
learn_weights = True
tradeoff_weight_reg = 0.
#tradeoff_weight_regs = 2**np.linspace(-3,3,7)
#tradeoff_weight_regs = 2**np.linspace(-1,1,3)
weighted_fitter = lsif_constructor.lsif_ratio_fitter(predictor_which_loss, num_basis, tradeoff_weight_reg, B_init_f_getter, learn_weights=learn_weights, learn_projection=learn_projection, num_tries=num_tries, many_optimizer_num_cycles=many_optimizer_num_cycles)
#fitter = cv.cv_fitter(fitter_list, cv_which_loss, cv_num_folds, cv_use_test)

In [8]:
# define random projection weighted fitter
#cv_which_loss = 'square'
#cv_num_folds = 3
#cv_num_folds = 2
#cv_use_test = False
predictor_which_loss = 'square'
num_basis = 100
#u_dim = 8
#pymanopt_options={'logverbosity':1, 'maxiter':10}
many_optimizer_num_cycles = 1
num_tries = 1
B_init_f_getter = None
learn_projection = False
learn_weights = True
random_projection = 1
tradeoff_weight_reg = 0.
#plot_b_info = constructors.simple_plot_b_info
plot_b_info = None
#tradeoff_weight_regs = 2**np.linspace(-3,3,7)
#tradeoff_weight_regs = 2**np.linspace(-1,1,3)
random_projection_weighted_fitter = lsif_constructor.lsif_ratio_fitter(predictor_which_loss, num_basis, tradeoff_weight_reg, B_init_f_getter, random_projection=random_projection, learn_weights=learn_weights, learn_projection=learn_projection, num_tries=num_tries, many_optimizer_num_cycles=many_optimizer_num_cycles, plot_b_info=plot_b_info)
#fitter = cv.cv_fitter(fitter_list, cv_which_loss, cv_num_folds, cv_use_test)

In [9]:
# define unweighted baseline
#cv_which_loss = 'square'
#cv_num_folds = 3
#cv_num_folds = 2
#cv_use_test = False
predictor_which_loss = 'square'
num_basis = 100
#u_dim = 8
#pymanopt_options={'logverbosity':1, 'maxiter':10}
many_optimizer_num_cycles = 1
num_tries = 1
B_init_f_getter = None
learn_projection = False
learn_weights = False
tradeoff_weight_reg = 0.
#tradeoff_weight_regs = 2**np.linspace(-3,3,7)
#tradeoff_weight_regs = 2**np.linspace(-1,1,3)
unweighted_fitter = lsif_constructor.lsif_ratio_fitter(predictor_which_loss, num_basis, tradeoff_weight_reg, B_init_f_getter, learn_weights=learn_weights, learn_projection=learn_projection, num_tries=num_tries, many_optimizer_num_cycles=many_optimizer_num_cycles)
#fitter = cv.cv_fitter(fitter_list, cv_which_loss, cv_num_folds, cv_use_test)

In [10]:
#
fitter_infos = [
#    ('mine', my_fitter),
#    ('unweighted', unweighted_fitter),
#    ('weighted', weighted_fitter),
    ('random_weighted', random_projection_weighted_fitter)
]

In [11]:
# run
run.run(random_data_getter, eval_num_trials, eval_which_loss, eval_num_folds, eval_use_test, fitter_infos)

0.0 tradeoff_weight_reg
try 0
0.289857952018 mean dist
[ 0.07044743  0.15218078  0.16850598  0.23069659  0.24823193  0.29555433
  0.31872519  0.36626732  0.54584633] sigma_range
0.0 tradeoff_weight_reg
try 0
0.335899642859 mean dist
[ 0.04890177  0.10338835  0.24951197  0.3152261   0.35594134  0.40485
  0.42696186  0.49036222  0.59677455] sigma_range
0.0 tradeoff_weight_reg
try 0
0.257260949545 mean dist
[ 0.01321377  0.07575375  0.10080053  0.12752123  0.17056059  0.26010226
  0.36299744  0.443347    0.53253268] sigma_range
('fitter: random_weighted, iteration 0: so far: 0.28(0.00)',)
0.0 tradeoff_weight_reg
try 0
0.349108447546 mean dist
[ 0.02736251  0.12644263  0.19561636  0.26136282  0.34686578  0.4099002
  0.45326751  0.53381607  0.60303374] sigma_range
0.0 tradeoff_weight_reg
try 0
0.338531363155 mean dist
[ 0.04086855  0.09593939  0.17325671  0.19786347  0.24882999  0.3254964
  0.43690061  0.59151667  0.75860643] sigma_range
0.0 tradeoff_weight_reg
try 0
0.288028349564 mean dis

,random_weighted
mean,0.266782
std,0.028362
